In [1]:
#pip install tqdm
from tqdm import tqdm


In [3]:
## Native Python library
import os

## 3rd party library imports
# stands for panel data -> Used for manipulating dataframes
import pandas as pd


## Graphical Subroutines
# Has a lot of plotting functionalities. Mimics matlab for numerical plotting
from matplotlib import pyplot as plt
from matplotlib.figure import Figure
import seaborn as sns

## Statistical Libraries
# One of the main statistical modelling libraries in Python.
# Mimics R in the callable functions and the attributes
import statsmodels
# API for calling different models
import statsmodels.api as sm
# API for calling models where the input to the model is in form of a formula like R
# E.g. the formula is y~x
import statsmodels.formula.api as smf
# Stands for SCIentific PYthon
# Has a lot of statistical hypothesis teste along with various probability distributions
from scipy import stats
import scipy.stats as sts
# Stands for NUMerical PYthon
# Has a lot of functions for both scalar and vector algebra
import numpy as np
# Library to run partial correlation
import pingouin as pg
# Library to generate various permutation and combination of different elements
import itertools
## sklearn imports
# Stands for Sci Kit LEARN
# Python's one of the most popular machine learning library containing various models
import sklearn
# Submodule used to split the data into training and test
from sklearn.model_selection import train_test_split
# Submodule used to scale the data
from sklearn.preprocessing import StandardScaler
# Submodule for linear regression
from sklearn.linear_model import LinearRegression
# Submodule for Recursive Feature Elimination and Recursive Feature Elimination with Cross Validation
from sklearn.feature_selection import RFE, RFECV
# Submodule that contains various model evaluation parameters
from sklearn import metrics


%matplotlib inline


In [17]:
model_data = pd.read_csv("/Users/brocktbennett/GitHub/Project Data/2023_TAMU_competition_data/Merged/New/cleaned_humana_inner9-29.csv") 
# Replace with #the appropriate user path - pd.read_csv('C:/Userpath/B2B.csv')
model_data = model_data[:100000]

/var/folders/_d/gcvc5_md1bg3809rkn2_lqk40000gn/T/ipykernel_7224/2024518884.py:1: DtypeWarning: Columns (11,12,13,14,17,87,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  model_data = pd.read_csv("/Users/brocktbennett/GitHub/Project Data/2023_TAMU_competition_data/Merged/New/cleaned_humana_inner9-29.csv") # Replace with #the appropriate user path - pd.read_csv('C:/Userpath/B2B.csv')


In [18]:
model_df = model_data[['X19', 
                       'X1', 'X2', 'X3', 'X4', 'X5',
                  'X6', 'X7', 'X8' ,'X9', 'X10', 'X11', 'X12','X13','X14',
                  'X15', 'X16', 'X17', 'X18']]

KeyError: "None of [Index(['X19', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',\n       'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18'],\n      dtype='object')] are in the [columns]"

In [14]:
## We need to do this as regression method in statsmodels in Python does not take Boolean inputs
## All the variables has to be numeric in nature

## Wrapper around the train_test split function for ease of usage and readability
## Splits the data in a 70-30 ratio according to the default parameters of this function
def split_data_check_target_variable_statistics(
    input_data: pd.DataFrame,
    target_variable_name: str,
    test_size_split_proportion: float = 0.3,
    random_state: int = 42,
) -> list[pd.DataFrame]:
    train, test = train_test_split(
        input_data,
        test_size=test_size_split_proportion,
        random_state=random_state,
        shuffle=True,
    )
    print("Descriptive Statistics for the training data")
    #display(train.drop('ID',axis=1).describe())
    print("Descriptive Statistics for the validation data")
    #display(test.drop('ID',axis=1).describe())

    train_reset_index = train.reset_index(drop=True)
    test_reset_index = test.reset_index(drop=True)

    return train_reset_index, test_reset_index

## Invoking the function
train, test = split_data_check_target_variable_statistics(
    input_data=model_data, target_variable_name="tgt_ade_dc_ind"
)

TypeError: 'type' object is not subscriptable

In [15]:
def _standard_scaling_training_test_df_return_scaled_df(
    training_data: pd.DataFrame, test_data: pd.DataFrame = pd.DataFrame()
) -> pd.DataFrame:
    """
    This function takes two dataframe in the order of training and validation.
    It applies the standard scaler function on the training set and transforms the test set using the mean and standard deviation of the training data.
    It ultimately returns two scaled dataframes. But if you do not pass a second dataset (validation data), it just returns the scaled version of the input data.
    """
    SS = StandardScaler()
    scaled_training_array = SS.fit_transform(training_data)
    scaled_training_df = pd.DataFrame(
        scaled_training_array, columns=training_data.columns
    )
    if len(test_data) != 0:
        scaled_test_array = SS.transform(test_data)
        scaled_test_df = pd.DataFrame(scaled_test_array, columns=test_data.columns)
        return scaled_training_df, scaled_test_df
    else:
        return scaled_training_df



In [13]:
train_scaled = _standard_scaling_training_test_df_return_scaled_df(train)
test_scaled = _standard_scaling_training_test_df_return_scaled_df(test)

NameError: name 'train' is not defined

In [24]:
## Wrapper around pandas method to subset the columns easily from a data frame
def _model_data_subset(input_data: pd.DataFrame, column_subset: list =None) -> pd.DataFrame:
    if column_subset is None:
        return input_data
    else:
        model_subset_data = input_data[column_subset]
        return model_subset_data

In [9]:

## Stepwise regression method where we give the p-value threshold
## and the best subset regression model is built
## NA values are dropped row-wise within this function so that Python does not have an issue executing the regression model
## The function starts to add features in a regression model
## depending on the cut-off points given
## The initial features are indexed and a regression is run and only lowest p-value which is less than the cut-off specified 
## Similar regressions are run for all the variables and only the new variables are considered and added if the p-value is less than the one specified in the function
## If the p-value increases beyond the cutoff after adding other variables, that variable is dropped
def _stepwise_selection_p_val(
    input_dataframe: pd.DataFrame,
    target_variable_name: str,
    column_subset: list = None,
    SL_in: float = 0.05,
    SL_out: float = 0.05,
) -> [pd.DataFrame, pd.Series, list]:
    model_subset_data = _model_data_subset(input_dataframe, column_subset)
    model_subset_data_nona = model_subset_data.dropna()

    target = model_subset_data_nona[target_variable_name]

    x_variables_subset_data = model_subset_data_nona.drop(target_variable_name, axis=1)

    initial_features = x_variables_subset_data.columns.tolist()
    best_features = []
    while len(initial_features) > 0:
        remaining_features = list(set(initial_features) - set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(
                target,
                sm.add_constant(model_subset_data_nona[best_features + [new_column]]),
            ).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if min_p_value < SL_in:
            best_features.append(new_pval.idxmin())
            while len(best_features) > 0:
                best_features_with_constant = sm.add_constant(
                    model_subset_data_nona[best_features]
                )
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if max_p_value >= SL_out:
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break
        else:
            break
    return x_variables_subset_data, target, best_features

In [25]:
# The above function should not be called on its own.
## The way to understand this is that functions starts wtih an underscore.
## The previous function is called into the public function build_linear_regression
## This function does the variable selection and returns a regression model summary output
def build_linear_regression_stepwise_based(
    input_dataframe: pd.DataFrame,
    target_variable_name: str,
    column_subset: list = None,
    SL_in: float = 0.05,
    SL_out: float = 0.05,
) -> [list, list]:
    """
    input_dataframe: Pass the dataframe on which you would want to build a regression model
    column_subset: The columns that you would want to consider for the best subset regression model. Please pass the target variable to the column subset as well if you are subsetting the data.
                    If nothing is passed, the entire data set is used. Be careful and do make sure that all the variable data types are compatible with the statsmodels.api regression model
    target_variable_name: Specify the name of the target variable as a string
    SL_in: The p-value for which the variable will be considered to enter the regression model. Default is 0.05
    SL_out: The p-value for which the variable will be considered to leave the regression model. Default is 0.05

    """

    x_variables_subset_data, target_variable, best_features = _stepwise_selection_p_val(
        input_dataframe, target_variable_name,column_subset, SL_in=SL_in, SL_out=SL_out
    )
    dataframe_subset_best_features = x_variables_subset_data[best_features]
    regression_model = sm.OLS(
        target_variable, sm.add_constant(dataframe_subset_best_features)
    ).fit()
    print("The best features are ", best_features)
    return regression_model.summary()

In [26]:
model_data_scaled_df =  _standard_scaling_training_test_df_return_scaled_df(model_data)
model_data_scaled_df

,Visit_Duration,MedProcess_Duration,ade_diagnosis,seizure_diagnosis,pain_diagnosis,fatigue_diagnosis,nausea_diagnosis,hyperglycemia_diagnosis,constipation_diagnosis,diarrhea_diagnosis,...,util_cat__ER,util_cat__IP_ACUTE,util_cat__IP_LTACH,util_cat__IP_MHSA,util_cat__IP_REHAB,util_cat__IP_SNF,util_cat__Outpatient,util_cat__Physician_Office,util_cat__Urgent_Care,cleaned_id
0,-1.519652,0.683938,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,1.553861,-0.258111,-0.060108,-0.278395
1,-1.519652,0.683938,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,1.553861,-0.258111,-0.060108,-0.278395
2,-1.519652,0.683938,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,1.553861,-0.258111,-0.060108,-0.278395
3,-1.519652,0.683938,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,1.553861,-0.258111,-0.060108,-0.278395
4,-1.519652,0.683938,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,1.553861,-0.258111,-0.060108,-0.278395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.482080,-0.519773,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,-0.643558,-0.258111,-0.060108,0.769292
99996,-0.482080,-0.519773,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,-0.643558,-0.258111,-0.060108,0.769292
99997,-0.482080,-0.519773,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,-0.643558,-0.258111,-0.060108,0.769292
99998,-0.482080,-0.519773,-0.234491,0.0,0.0,-0.137517,-0.153705,-0.101212,-0.10101,-0.068718,...,-0.158412,-0.294904,0.0,0.0,0.0,-0.117113,-0.643558,-0.258111,-0.060108,0.769292


In [16]:
SL_in = 0.05 # entry p-value
SL_out = 0.05 # exit p-value

build_linear_regression_stepwise_based(model_data_scaled_df, 'tgt_ade_dc_ind', None,0.05,0.05)


NameError: name 'build_linear_regression_stepwise_based' is not defined